In [16]:
from __future__ import print_function, division
#import example_helper
import json
import csv
import numpy as np
from deepmoji.sentence_tokenizer import SentenceTokenizer
from deepmoji.model_def import deepmoji_emojis
from deepmoji.global_variables import PRETRAINED_PATH, VOCAB_PATH
import pandas as pd
OUTPUT_PATH = 'test_sentences.csv'

TEST_SENTENCES = [u'I love mom\'s cooking',
                  u'I love how you never reply back..',
                  u'I love cruising with my homies',
                  u'I love messing with yo mind!!',
                  u'I love you and now you\'re just gone..',
                  u'Paul is dead',
                  u'I am very afraid of dying',
                  u'I am very happy right now!']



ImportError: No module named pandas

In [15]:
df_train = pd.read_csv("data/train.csv")
df_test  = pd.read_csv("data/test.csv")

NameError: name 'pd' is not defined

In [13]:

def top_elements(array, k):
    ind = np.argpartition(array, -k)[-k:]
    return ind[np.argsort(array[ind])][::-1]


maxlen = 30
batch_size = 32

print('Tokenizing using dictionary from {}'.format(VOCAB_PATH))
with open(VOCAB_PATH, 'r') as f:
    vocabulary = json.load(f)
st = SentenceTokenizer(vocabulary, maxlen)
tokenized, _, _ = st.tokenize_sentences(TEST_SENTENCES)

print('Loading model from {}.'.format(PRETRAINED_PATH))
model = deepmoji_emojis(maxlen, PRETRAINED_PATH)
model.summary()


Tokenizing using dictionary from /home/wilhelm/INSA/5A/HDDL/Projet_HDDL/HDDL_moji/model/vocabulary.json
Loading model from /home/wilhelm/INSA/5A/HDDL/Projet_HDDL/HDDL_moji/model/deepmoji_weights.hdf5.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 30, 256)      12800000    input_4[0][0]                    
__________________________________________________________________________________________________
activation_4 (Activation)       (None, 30, 256)      0           embedding[0][0]                  
__________________________________________________________________________________________________
bi_lstm

In [14]:

print('Running predictions.')
prob = model.predict(tokenized)

# Find top emojis for each sentence. Emoji ids (0-63)
# correspond to the mapping in emoji_overview.png
# at the root of the DeepMoji repo.
print('Writing results to {}'.format(OUTPUT_PATH))
scores = []
for i, t in enumerate(TEST_SENTENCES):
    t_tokens = tokenized[i]
    t_score = [t]
    t_prob = prob[i]
    ind_top = top_elements(t_prob, 5)
    t_score.append(sum(t_prob[ind_top]))
    t_score.extend(ind_top)
    t_score.extend([t_prob[ind] for ind in ind_top])
    scores.append(t_score)
    print(t_score)

with open(OUTPUT_PATH, 'wb') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', lineterminator='\n')
    writer.writerow(['Text', 'Top5%',
                     'Emoji_1', 'Emoji_2', 'Emoji_3', 'Emoji_4', 'Emoji_5',
                     'Pct_1', 'Pct_2', 'Pct_3', 'Pct_4', 'Pct_5'])
    for i, row in enumerate(scores):
        try:
            writer.writerow(row)
        except Exception:
            print("Exception at row {}!".format(i))

Running predictions.
Writing results to test_sentences.csv
[u"I love mom's cooking", 0.66769218631088734, 36, 4, 8, 16, 47, 0.49053311, 0.087657787, 0.030925233, 0.029632866, 0.028943187]
[u'I love how you never reply back..', 0.39060431718826294, 1, 19, 55, 25, 46, 0.13978004, 0.082515739, 0.062788285, 0.054191161, 0.051329087]
[u'I love cruising with my homies', 0.5413312129676342, 31, 6, 30, 15, 13, 0.33965704, 0.066058099, 0.057058834, 0.040674038, 0.037883203]
[u'I love messing with yo mind!!', 0.48666664958000183, 54, 44, 9, 50, 49, 0.17236328, 0.1184359, 0.079605907, 0.06371031, 0.052551255]
[u"I love you and now you're just gone..", 0.67333512753248215, 46, 5, 27, 35, 34, 0.39116541, 0.11033387, 0.073462442, 0.052958723, 0.045414682]
[u'Paul is dead', 0.54029577970504761, 34, 46, 51, 3, 35, 0.19904983, 0.12621364, 0.07727541, 0.074507423, 0.063249476]
[u'I am very afraid of dying', 0.51727710664272308, 52, 39, 43, 34, 27, 0.16364294, 0.10124379, 0.094967499, 0.08395201, 0.07347